### basic chat stream

In [1]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import StructuredTool, ToolException
from typing import List, Annotated
from langchain.agents import load_tools
from langchain_experimental.utilities import PythonREPL
from langgraph.prebuilt import ToolNode


NameError: name 'llm' is not defined

In [3]:
import fitz
import os

In [7]:
d = fitz.open(os.path.join('/home/aditya-ladawa/Aditya/z_projects/aireas/api/static/files/', 'a.pdf'))

In [14]:
os.getcwd()

'/home/aditya-ladawa/Aditya/z_projects/aireas/api'

In [67]:
import os
import xml.etree.ElementTree as ET

# Specify the path to your XML file
file_path = os.path.join('metas', 'a.pdf.tei.xml')

# Parse the XML file
tree = ET.parse(file_path)
root = tree.getroot()

# Now you can iterate through elements or access specific tags
for element in root.iter():
    if element.text == None or element.text!= 'None':
        print(element.text)




	

		

			

				
Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need

				
None

					
None
2 Aug 2023

				

					

						

							
None
Ashish
Vaswani
avaswani@google.com

								
Google Brain Google Research Google Research Google Research University of Toronto Google Brain 31st Conference on Neural Information Processing Systems (NIPS 2017) , Long Beach , CA , USA.
Google Brain
Google Research
Google Research
Google Research
Google Brain
31st Conference on Neural Information Processing Systems (NIPS 2017)
University of Toronto

									
Long Beach
CA
USA

							
None
Noam
Shazeer

								
Google Brain Google Research Google Research Google Research University of Toronto Google Brain 31st Conference on Neural Information Processing Systems (NIPS 2017) , Long Beach , CA , USA.
Google Brain
Google Research
Google Research
Google Resea

In [59]:
import xml.etree.ElementTree as ET

# Load the XML file
file_path = os.path.join('metas', 'a.pdf.tei.xml')
tree = ET.parse(file_path)
root = tree.getroot()

# Define the TEI namespace
namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

# Function to extract text safely
def get_text_or_none(element, tag):
    elem = element.find(tag, namespace)
    return elem.text if elem is not None else None

# Extract title
title = get_text_or_none(root, './/tei:titleStmt/tei:title')
print("Title:", title)

# Extract publication date
date = get_text_or_none(root, './/tei:publicationStmt/tei:date')
print("Publication Date:", date)

# Extract authors
authors = []
for author in root.findall('.//tei:biblStruct/tei:analytic/tei:author', namespace):
    forename = get_text_or_none(author, 'tei:persName/tei:forename')
    surname = get_text_or_none(author, 'tei:persName/tei:surname')
    authors.append(f"{forename} {surname}")
print("Authors:", authors)

# Extract abstract (if present)
abstract = get_text_or_none(root, './/tei:teiHeader/tei:profileDesc/tei:abstract')
print("Abstract:", abstract)


Title: Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need
Publication Date: 2 Aug 2023
Authors: ['Ashish Vaswani', 'Noam Shazeer', 'Google Brain', 'Niki Parmar', 'Jakob Uszkoreit', 'Llion Jones', 'Aidan Gomez', 'Łukasz Kaiser', 'Chris Dyer', 'Adhiguna Kuncoro', 'Miguel Ballesteros', 'Noah Smith', 'Kaiming He', 'Xiangyu Zhang', 'Shaoqing Ren', 'Jian Sun', 'Sepp Hochreiter', 'Jürgen Schmidhuber', 'Zhongqiang Huang', 'Mary Harper', 'Łukasz Kaiser', 'Samy Bengio', 'Łukasz Kaiser', 'Ilya Sutskever', 'Yoon Kim', 'Carl Denton', 'Luong Hoang', 'Alexander Rush', 'Diederik Kingma', 'Jimmy Ba', 'Mary Mitchell P Marcus', 'Ann Marcinkiewicz', 'Beatrice Santorini', 'David Mcclosky', 'Eugene Charniak', 'Mark Johnson', 'Ankur Parikh', 'Oscar Täckström', 'Dipanjan Das', 'Jakob Uszkoreit', 'Slav Petrov', 'Leon Barrett', 'Romain Thibaux', 'Dan Klein', 'Nitis

In [61]:
from lxml import etree

def extract_sections_and_text(element, namespaces):
    sections = {}
    for child in element:
        # Check for section-like elements (like div, p, section, etc.)
        if child.tag in ['{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}p', '{http://www.tei-c.org/ns/1.0}section']:
            section_name = child.find('{http://www.tei-c.org/ns/1.0}head', namespaces)  # Assuming section names are in <head> tags
            section_name = section_name.text if section_name is not None else "Unnamed Section"
            sections[section_name] = []

            # Collect text from the section
            for para in child.xpath('.//tei:p', namespaces):
                sections[section_name].append(para.text)

    return sections

# Parse the XML file
tree = etree.parse(file_path)
root = tree.getroot()

# Define namespaces if needed
namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}

# Extract sections and text
all_sections = extract_sections_and_text(root, namespaces)

# Output the extracted information
for section, texts in all_sections.items():
    print(f"Section: {section}")
    for text in texts:
        print(text)


# Adaptive RAG

In [50]:
from dotenv import load_dotenv

load_dotenv()

True

In [51]:
from langchain_groq import ChatGroq
from pprint import pprint
llm = ChatGroq(model='llama-3.1-70b-versatile')


In [63]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
        
    )

structured_llm_router = llm.with_structured_output(RouteQuery)
routing_system_prompt = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains vectors of pdf texts of several research papers. Currently these pdfs are in the vector database: ['Attention is all you need', 'MaGVIT: Mased Generative Video transformer', 'SAN: INDUCING METRIZABILITY OF GAN WITH
DISCRIMINATIVE NORMALIZED LINEAR LAYER'].
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", routing_system_prompt),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
# print(
#     question_router.invoke(
#         {"question": "Who will the Bears draft first in the NFL draft?"}
#     )
# )
print(question_router.invoke({"question": 'give me the latest updates on Magvit paper?'}))

datasource='vectorstore'


In [65]:
# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )
structured_llm_doc_grader = llm.with_structured_output(GradeDocuments)

from team_tools import Qretriever

retriever = Qretriever

# Prompt
doc_grader_system_prompt = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", doc_grader_system_prompt),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

question = 'give me the latest updates on Magvit paper, what are other sttae of the art models?'
retrieval_grader = grade_prompt | structured_llm_doc_grader
docs = retriever.invoke(question)
doc_txt = docs

print(retrieval_grader.invoke({"question": question, "document": doc_txt}))


binary_score='yes'


In [59]:
docs

[Document(metadata={'pdf_id': 'a.pdf', 'score': 0.5841408}, page_content='the verb ‘making’, completing the phrase ‘making...more difficult’. Attentions here shown only for\nthe word ‘making’. Different colors represent different heads. Best viewed in color.\n13\nInput-Input Layer5\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nInput-Input Layer5\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nThe\nLaw\nwill\nnever\nbe\nperfect\n,\nbut\nits\napplication\nshould\nbe\njust\n-\nthis\nis\nwhat\nwe\nare\nmissing\n,\nin\nmy\nopinion\n.\n<EOS>\n<pad>\nFigure 4: Two attention heads, also in layer 5 of 6, apparently involved in anap

In [61]:

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")


In [62]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

        
# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
pprint(generation)

("I don't know how to explain 'attention' in a way that a 5-year-old can "
 'understand, based on the provided context.')


In [8]:
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("What is attention")

NameError: name 'format_docs' is not defined

In [52]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )
structured_llm_hallucinations_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
hallucination_grader_system_prompt = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", hallucination_grader_system_prompt),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_hallucinations_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

In [53]:
# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )
structured_llm_answer_grader = llm.with_structured_output(GradeAnswer)

# Prompt
answer_grader_system_prompt = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", answer_grader_system_prompt),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_answer_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

In [38]:

# Prompt
rewriter_system_prompt = """Rewrite the question to optimize it for retrieval based on its semantic intent."""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rewriter_system_prompt),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'Here\'s an improved version of the question:\n\n"What are the different methods for transferring video content, including analog and digital formats, from one device or medium to another?"\n\nThis revised question provides more context and specificity, which can help improve retrieval based on its semantic intent. \n\nAlternatively, you could also ask:\n\n* "What video transfer methods are available for converting, copying, or moving video content?"\n* "What are the various techniques used for video migration, duplication, or conversion between different devices or formats?"\n* "How can video content be transferred from one device or medium to another, and what are the different techniques involved?"'

In [10]:
from team_tools import tavily_search_tool

web_search_tool = tavily_search_tool

In [42]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [ ]:
pprint(docs)

In [43]:
from langchain.schema import Document


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": [web_results], "question": question}


### Edges ###


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [44]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [ ]:
from pprint import pprint

# Run
inputs = {
    "question": "in attention is all you need paper, what is positional encoding?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint(value, indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

# Multi query retriever

In [40]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from pprint import pprint 

load_dotenv()

llm = ChatGroq(model='llama-3.2-90b-vision-preview')

In [55]:
from team_tools import Qretriever
from langchain_core.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List


from langchain_core.messages import AIMessage, AIMessageChunk
def parse(ai_message: AIMessage) -> List[str]:
    """Parse the AI message."""
    # Split the input string by newline, strip whitespace and single quotes, and return a list of queries
    return [line for line in ai_message.content.split('\n')]


# Define the query prompt
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Break down complex queries into simpler ones. Please adhere to the context and topic of the original question. Rephrase the simpler queries to make them efficient for querying a vector database.
    Please try to only generate the most minimum/least number of queries required to answer the question. Do not add repetitive or irrelevant questions.
ONLY PROVIDE THE OUTPUT. List the questions seperated by newlines.

Example input: What is attention mechanism? How can i explain it to a child?
Output:
Explain attention mechanism in deep learning.
Give an example to explain attention mechanism to 5 year old.

Original question: {question}

    """
)


# Initialize the LLM chain
llm_chain = QUERY_PROMPT | llm | parse

question = "'give me the latest updates on Magvit paper"

pprint(llm_chain.invoke(question))

# Initialize the MultiQueryRetriever
retriever = MultiQueryRetriever(
    retriever=Qretriever, llm_chain=llm_chain
)

# Run the retriever
results = retriever.invoke(question)


# Assuming `documents` is a list of Document objects
refines = [{ 'pdf_id': doc.metadata['pdf_id'], 'score': doc.metadata['score'], 'page_content': doc.page_content } for doc in results]

print(refines)
print('\n')

llm.invoke(f'{question}\n{results}')

['What is the Magvit paper about?',
 'When was the Magvit paper last updated?',
 'What are the key findings or contributions of the latest Magvit paper '
 'update?']
[{'pdf_id': 'm.pdf', 'score': 0.6426568, 'page_content': '33.9\n34.4\n21.5\n22.1\n26.0\n20.7\n20.4\nMAGVIT-L\n20.6\n26.7\n21.6\n45.5\n30.9\n19.9\n15.3\n14.5\n20.2\n12.0\n14.7\nTable 13. Generation performance on NuScenes, Objectron, and Web videos evaluated by FVD.\ning videos of UCF-101, or 13.3K training and testing videos\nof UCF-101 for those marked with #. Fig. 12 provides a vi-\nsual comparison to the baseline methods.\nAs shown, even the smaller MAGVIT-B performs favor-\nably against previous state-of-the-art model TATS [13] by a\nlarge margin. MAGVIT-L pushes both the FVD (332 →\n76, ↓77%) and IS (79.28 →89.27, ↑13%) to a new\nlevel, while outperforming the contemporary work Make-\nA-Video [29] which is pretrained on significantly large ex-\ntra training data.\nFrame prediction.\nFor the frame prediction task on\nB

AIMessage(content='MAGVIT (MAGVIT-B and MAGVIT-L) is a state-of-the-art model for video generation tasks, including class-conditional generation and frame prediction. Here are the latest updates on MAGVIT paper:\n\n**Key Findings:**\n\n1. MAGVIT outperforms previous state-of-the-art models, including TATS and Make-A-Video, on various video generation benchmarks.\n2. Even the smaller MAGVIT-B model performs favorably against previous state-of-the-art models.\n3. MAGVIT-L pushes the FVD (Fréchet Video Distance) and IS (Inception Score) to a new level, outperforming contemporary work Make-A-Video, which is pre-trained on significantly large extra training data.\n4. MAGVIT generates higher-quality frames with substantial motion, outperforming baseline models.\n\n**Evaluation Results:**\n\n1. Table 13 shows the generation performance of MAGVIT on NuScenes, Objectron, and Web videos evaluated by FVD.\n2. Table 1 shows the class-conditional generation results on UCF-101, where MAGVIT surpasse

# Qdrant for fastapi

In [ ]:

# # Qdrant client
# @app.on_event("startup")
# async def startup_event():
#     client = connect_to_qdrant()
#     app.state.qdrant_client = client

# @app.on_event("shutdown")
# async def shutdown_event():
#     client = app.state.qdrant_client
#     if client:
#         print("Shutting down Qdrant client")
#         client.close()

# # Qdrant client
# def get_qdrant_client():
#     return app.state.qdrant_client
